In [74]:
import json
import os

In [75]:
weekday_schedules = []
weekend_schedules = []
activity_map = {
"bathe_shower" : "showering",
"brush_teeth" : "brushing_teeth",
"computer_work" : "computer_work",
"connect_w_friends" : None,
"clean" : "cleaning",
"clean_kitchen" : "kitchen_cleaning",
"diary_journaling" : None,
"do_laundry" : "laundry",
"get_dressed" : "getting_dressed",
"hand_wash_clothes" : "laundry",
"leave_home" : None,
"come_home" : None,
"listen_to_music" : "listening_to_music",
"prepare_eat_breakfast" : "breakfast",
"prepare_eat_dinner" : "dinner",
"prepare_eat_lunch" : "lunch",
"play_music" : "playing_music",
"read" : "reading",
"sleep" : "sleeping",
"take_medication" : "taking_medication",
"take_out_trash" : "take_out_trash",
"use_restroom" : "going_to_the_bathroom",
"vacuum_clean" : "vaccuum_cleaning",
"wash_dishes" : "wash_dishes",
"watch_tv" : "watching_tv",
"wake_up" : "getting_out_of_bed",
}
start_times = [6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]

In [76]:
def get_schedule(data):
    activity_times = {}
    for timestring,activities in data.items():
        start_time = timestring.split('m')[0]
        start_time = int(start_time[:-1]) if start_time[-1] == 'a' else int(start_time[:-1])+12
        if start_time == 24:
            start_time = 12
        for act in activities:
            if act in activity_times:
                activity_times[act].append(start_time)
            else:
                activity_times[act] = [start_time]
    schedule = {}
    for activity, start_times in activity_times.items():
        sch_activity = activity_map[activity]
        if sch_activity is None:
            continue
        schedule[sch_activity] = [[start_times[0], start_times[0]+1]]
        for time in start_times[1:]:
            if schedule[sch_activity][-1][1] == time:
                schedule[sch_activity][-1][1] = time+1
            else:
                schedule[sch_activity].append([time, time+1])
    return schedule

In [77]:
for root, dirs, files in os.walk('data/AMT_Schedules'):
    for f in files:
        if f.endswith('.json'):
            with open(os.path.join(root,f)) as file:
                sch = json.load(file)
            with open(os.path.join('data/sourcedSchedules/weekday',f), 'w') as f:
                json.dump(get_schedule(sch['activitiesChosen']), f)
